In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE107105"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE107105"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE107105.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE107105.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE107105.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray analysis of freshly isolated synovial fibroblast subsets in patients with rheumatoid arthritis or osteoarthritis"
!Series_summary	"Transcriptomics of distinct subpopulations of synovial fibroblasts from osteoarthritis and rheumatoid arthritis arthroplasty tissues."
!Series_overall_design	"We gated 7 subsets of synovial fibroblasts by protein surface markers and used microarrays to profile gene expression of cells taken from each gate. This data includes 37 samples from 3 osteoarthritis donors and 3 rheumatoid arthritis donors."
Sample Characteristics Dictionary:
{0: ['disease: OA', 'disease: RA'], 1: ['age: 59', 'age: 78', 'age: 86', 'age: 51', 'age: 79', 'age: 48'], 2: ['Sex: Female', 'Sex: Male'], 3: ['joint: Knee', 'joint: MP'], 4: ['surgery_date: 2014-04-25', 'surgery_date: 2014-07-18', 'surgery_date: 2014-09-26', 'surgery_date: 2014-04-08', 'surgery_date: 2014-06-12', 'surgery_date: 2014-10-20'], 5: ['microarray_date: 2014-06-19', 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import re
import pandas as pd
import numpy as np
import json
from typing import Dict, Any, List, Callable, Optional, Union, Tuple

# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset appears to contain microarray data
# which is a form of gene expression data, not just miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Osteoarthritis), we can see "disease: OA" and "disease: RA" in row 0
# where OA = Osteoarthritis and RA = Rheumatoid Arthritis
trait_row = 0

# For age, we can see values like "age: 59", "age: 78", etc. in row 1
age_row = 1

# For gender, we can see "Sex: Female" and "Sex: Male" in row 2
gender_row = 2

# 2.2 Data Type Conversion

def convert_trait(value: str) -> Optional[int]:
    """
    Convert trait value to binary (0 for non-Osteoarthritis, 1 for Osteoarthritis)
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.upper() == "OA" or "OSTEOARTHRITIS" in value.upper():
        return 1
    elif value.upper() == "RA" or "RHEUMATOID" in value.upper():
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age value to continuous numeric value
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to convert to float
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender value to binary (0 for female, 1 for male)
    """
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.upper() in ["FEMALE", "F"]:
        return 0
    elif value.upper() in ["MALE", "M"]:
        return 1
    else:
        return None

# 3. Save Metadata
# Based on trait_row being not None, trait data is available
is_trait_available = trait_row is not None

# Initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this section as we don't have access to the clinical_data yet
# We'll need to wait for the next steps where the data is fully loaded
# This information will be available in subsequent processing steps

print(f"Initial evaluation complete for cohort: {cohort}")
print(f"- Gene expression data available: {is_gene_available}")
print(f"- Trait data available: {is_trait_available}")
print(f"- trait_row: {trait_row}")
print(f"- age_row: {age_row}")
print(f"- gender_row: {gender_row}")


Initial evaluation complete for cohort: GSE107105
- Gene expression data available: True
- Trait data available: True
- trait_row: 0
- age_row: 1
- gender_row: 2


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE107105/GSE107105_series_matrix.txt.gz


Gene data shape: (53617, 36)
First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These gene identifiers appear to be probe IDs rather than human gene symbols
# They are numerical identifiers (like '16650001') rather than standard gene symbols (like 'BRCA1', 'TP53', etc.)
# These are likely platform-specific probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'RANGE_STRAND', 'RANGE_START', 'RANGE_END', 'total_probes', 'GB_ACC', 'SPOT_ID', 'RANGE_GB']
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}

Searching for platform information in SOFT file:
!Series_platform_id = GPL16686

Searching for gene symbol information in SOFT file:
Found references to gene symbols:
!Platform_relation = Alternative to: GPL25483 (Gene symbol version)

Checking for additional annotation files in the directory:
[

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene identifiers in the gene expression data and gene annotation data
# The platform ID is GPL16686, which appears to be a custom platform
# We need to explore the annotation data more carefully to find usable gene identifiers

# Let's look at more rows of the annotation to better understand the data structure
print("Exploring additional annotation columns...")

# Check which columns might contain gene-related information
for col in gene_annotation.columns:
    non_null_count = gene_annotation[col].notna().sum()
    total_count = len(gene_annotation)
    print(f"Column '{col}': {non_null_count}/{total_count} non-null values ({non_null_count/total_count:.2%})")

# For this platform (GPL16686), we need to check if the SPOT_ID column contains genomic locations
# that can be linked to gene identifiers
spot_id_examples = gene_annotation['SPOT_ID'].dropna().head(10).tolist()
print(f"SPOT_ID examples: {spot_id_examples}")

# Let's check if we can extract any gene-related information from the annotation
sample_gb_acc = gene_annotation['GB_ACC'].dropna().head(10).tolist()
print(f"Sample GB_ACC values: {sample_gb_acc}")

# Since the platform appears to be from a custom transcript-based approach, 
# we need to use both ID and GB_ACC columns
prob_col = 'ID'
gene_col = 'GB_ACC'

# Create the mapping dataframe (for accession numbers)
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping shape: {mapping_df.shape}")
print("Sample of gene mapping data:")
print(mapping_df.head())

# Check how many probes have gene annotations
total_probes = len(gene_data.index)
mapped_probes = len(mapping_df)
print(f"Total probes in expression data: {total_probes}")
print(f"Probes with gene annotations: {mapped_probes} ({mapped_probes/total_probes:.2%})")

# Since the mapping data contains accession numbers instead of gene symbols,
# let's modify the mapping before applying it
# Let's first check if we have any gene symbols in the mapping
sample_genes = mapping_df['Gene'].apply(extract_human_gene_symbols).head(10).tolist()
print(f"Extracted gene symbols from mapping: {sample_genes}")

# Since we're dealing with a custom array with limited gene annotations,
# I'll augment the mapping process to keep the original accession numbers
# This might not give us standard gene symbols, but it will preserve the data
# for downstream analysis
def custom_extract_identifiers(text):
    """Extract gene identifiers including accession numbers like NR_/XR_ series"""
    if not isinstance(text, str):
        return []
    # Keep accession numbers intact
    return [text]

# Create a modified version of the mapping dataframe that keeps accession numbers
mapping_df_modified = mapping_df.copy()
mapping_df_modified['Gene'] = mapping_df_modified['Gene'].apply(custom_extract_identifiers)

# Apply gene mapping with the modified approach
gene_data_with_acc = gene_data.copy()
# Remove the index name to avoid conflicts during the join operation
gene_data_with_acc.index.name = None

# Perform a simpler mapping for this dataset
# Convert expression matrix to have probe IDs as a column instead of index
expression_df_reset = gene_data.reset_index()
# Merge with mapping data
merged_df = pd.merge(expression_df_reset, mapping_df, on='ID', how='inner')
# Remove rows with missing gene information
merged_df = merged_df.dropna(subset=['Gene'])
# Extract sample columns (all columns except ID and Gene)
sample_cols = [col for col in merged_df.columns if col not in ['ID', 'Gene']]
# Group by Gene and calculate the mean expression for each gene
gene_data = merged_df.groupby('Gene')[sample_cols].mean()

print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(list(gene_data.index[:10]))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Exploring additional annotation columns...
Column 'ID': 1984229/1984229 non-null values (100.00%)
Column 'RANGE_STRAND': 1982020/1984229 non-null values (99.89%)
Column 'RANGE_START': 51772/1984229 non-null values (2.61%)
Column 'RANGE_END': 51772/1984229 non-null values (2.61%)
Column 'total_probes': 51772/1984229 non-null values (2.61%)
Column 'GB_ACC': 17623/1984229 non-null values (0.89%)
Column 'SPOT_ID': 53981/1984229 non-null values (2.72%)
Column 'RANGE_GB': 47833/1984229 non-null values (2.41%)


SPOT_ID examples: ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581', 'chr1:329790-342507', 'chr1:367640-368634', 'chr1:459656-461954', 'chr1:523009-532878', 'chr1:557143-566063']
Sample GB_ACC values: ['NR_046018', 'NR_024368', 'NR_029406', 'XR_132471', 'NR_047526', 'NM_152486', 'NM_198317', 'NM_005101', 'NM_198576', 'NR_038869']


Gene mapping shape: (17623, 2)
Sample of gene mapping data:
          ID       Gene
0   16657436  NR_046018
4   16657450  NR_024368
7   16657476  NR_029406
9   16657485  XR_132471
15  16657514  NR_047526
Total probes in expression data: 53617
Probes with gene annotations: 17623 (32.87%)
Extracted gene symbols from mapping: [[], [], [], [], [], [], [], [], [], []]
Gene expression data shape after mapping: (16346, 36)
First few gene symbols after mapping:
['NM_000014', 'NM_000015', 'NM_000019', 'NM_000020', 'NM_000021', 'NM_000022', 'NM_000025', 'NM_000026', 'NM_000027', 'NM_000030']


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE107105.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (16346, 36)
Gene data column names (sample IDs):
Index(['GSM2861345', 'GSM2861346', 'GSM2861347', 'GSM2861348', 'GSM2861349'], dtype='object')

Raw clinical data structure:
Clinical data shape: (19, 37)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM2861345', 'GSM2861346', 'GSM2861347',
       'GSM2861348'],
      dtype='object')

Sample characteristics dictionary:
{0: ['disease: OA', 'disease: RA'], 1: ['age: 59', 'age: 78', 'age: 86', 'age: 51', 'age: 79', 'age: 48'], 2: ['Sex: Female', 'Sex: Male'], 3: ['joint: Knee', 'joint: MP'], 4: ['surgery_date: 2014-04-25', 'surgery_date: 2014-07-18', 'surgery_date: 2014-09-26', 'surgery_date: 2014-04-08', 'surgery_date: 2014-06-12', 'surgery_date: 2014-10-20'], 5: ['microarray_date: 2014-06-19', 'microarray_date: 2014-12-01', 'microarray_date: 2014-08-04', 'microarray_date: 2014-12-18'], 6: ['cd34_protein: -', 'cd34_protein: +', 'cd34_protein: NA'], 7: ['thy1_protein: -', 'thy1_protein: +', 'thy1_protei

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (36, 16347)
For the feature 'Osteoarthritis', the least common label is '1' with 15 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Osteoarthritis/cohort_info.json


Linked data saved to ../../output/preprocess/Osteoarthritis/GSE107105.csv
